This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Neural style transfer

### The content loss

### The style loss

### Neural style transfer in Keras

**Getting the style and content images**

In [1]:

from tensorflow import keras

base_image_path = keras.utils.get_file(
    "sf.jpg", origin="https://img-datasets.s3.amazonaws.com/sf.jpg")
style_reference_image_path1 = keras.utils.get_file(
    "starry_night.jpg", origin="https://img-datasets.s3.amazonaws.com/starry_night.jpg")

original_width, original_height = keras.utils.load_img(base_image_path).size
img_height = 400
img_width = round(original_width * img_height / original_height)

In [12]:
import cv2
import os

def reset_folder(folder_path):
    if os.path.exists(folder_path):
        # Delete the folder and its contents
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)
            if os.path.isdir(item_path):
                # Recursively delete subfolders and files
                reset_folder(item_path)
            else:
                os.remove(item_path)
        os.rmdir(folder_path)

    # Recreate the folder
    os.makedirs(folder_path)

def video_to_frames(mp4_path, frame_folder):
    reset_folder(frame_folder)
    vidcap = cv2.VideoCapture(mp4_path)
    success, image = vidcap.read()
    count = 0

    while success:
        frame_path = os.path.join(frame_folder, f"frame{count:04d}.jpg")  # Save frame with zero-padded numbers
        cv2.imwrite(frame_path, image)
        success, image = vidcap.read()
        print('Read a new frame: ', success)
        count += 1

    # Sort the frames in the folder
    frame_files = sorted([f for f in os.listdir(frame_folder) if f.endswith('.jpg')])

    for i, frame_file in enumerate(frame_files):
        new_frame_path = os.path.join(frame_folder, f"frame{i:04d}.jpg")  # Rename the frames
        print("renaming: %s to: %s" % (frame_file, new_frame_path))
        os.rename(os.path.join(frame_folder, frame_file), new_frame_path)
        


def frames_to_video(frame_folder, output_video_path, frame_rate):
    frame_files = [f for f in os.listdir(frame_folder) if f.endswith('.jpg')]
    frame_files.sort(key=lambda x: int(x[5:-4]))  # Sort the frames in the correct order

    frame = cv2.imread(os.path.join(frame_folder, frame_files[0]))
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    for frame_file in frame_files:
        print(f"putting: {frame_file} into the video")
        frame_path = os.path.join(frame_folder, frame_file)
        frame = cv2.imread(frame_path)
        out.write(frame)

    out.release()

**Auxiliary functions**

In [3]:
import numpy as np

def preprocess_image(image_path):
    img = keras.utils.load_img(
        image_path, target_size=(img_height, img_width))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.vgg19.preprocess_input(img)
    return img

def deprocess_image(img):
    img = img.reshape((img_height, img_width, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype("uint8")
    return img

**Using a pretrained VGG19 model to create a feature extractor**

In [4]:
model = keras.applications.vgg19.VGG19(weights="imagenet", include_top=False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

**Content loss**

In [5]:
def content_loss(base_img, combination_img):
    return tf.reduce_sum(tf.square(combination_img - base_img))

**Style loss**

In [6]:
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style_img, combination_img):
    S = gram_matrix(style_img)
    C = gram_matrix(combination_img)
    channels = 3
    size = img_height * img_width
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

**Total variation loss**

In [7]:
def total_variation_loss(x):
    a = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, 1:, : img_width - 1, :]
    )
    b = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, : img_height - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))

**Defining the final loss that you'll minimize**

In [8]:
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
content_layer_name = "block5_conv2"
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=())
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(
          style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * style_loss_value

    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

**Setting up the gradient-descent process**

In [9]:
import tensorflow as tf
import keras as keras
import os
from keras.optimizers import SGD
# from google.colab import files  # Import the files module

# Define a custom loss function that computes the total variation loss
@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.5)

def process_frames(original_frames_folder, processed_frames_folder):
    # Specify the directory containing your images
    reset_folder(processed_frames_folder)

    # List all files in the directory
    file_list = os.listdir(original_frames_folder)

    # Filter for image files (e.g., JPEG or PNG)
    file_list.sort()

    for i, frame in enumerate(file_list[:5]):
        print(frame)
        base_image = preprocess_image(os.path.join(original_frames_folder, frame))
        style_reference_image = preprocess_image(style_reference_image_path1)
        combination_image = tf.Variable(preprocess_image(os.path.join(original_frames_folder, frame)))
        iterations = 3
        for j in range(1, iterations + 1):
            loss, grads = compute_loss_and_grads(
                combination_image, base_image, style_reference_image
            )
            optimizer.apply_gradients([(grads, combination_image)])
            if j % 1 == 0:
                print(f"Iteration {j}: loss={loss:.2f}")
                img = deprocess_image(combination_image.numpy())
                fname = os.path.join(processed_frames_folder, f"ProcessedFrame-{i+1:04d}.png")
                cv2.imwrite(fname, img)  # Save the processed frame with a formatted filename
                print(f"Saving {fname} to {processed_frames_folder}")


In [10]:
import os
import cv2

def frames_to_video(frame_folder, output_video_path, frame_rate):
    frame_files = [f for f in os.listdir(frame_folder) if f.endswith('.png')]
    frame_files.sort(key=lambda x: int(x.split('-')[-1][:-4]))  # Sort the frames based on the numeric part

    frame = cv2.imread(os.path.join(frame_folder, frame_files[0]))
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    for frame_file in frame_files:
        frame_path = os.path.join(frame_folder, frame_file)
        frame = cv2.imread(frame_path)
        out.write(frame)

    out.release()

# # Example usage:
# frame_folder = '/content/ProcessedFrames'
# output_video_path = '/content/output.mp4'
# frame_rate = 2
# frames_to_video(frame_folder, output_video_path, frame_rate)


In [ ]:
#MAIN


#Extracting frames
input_video= r"C:\Users\madsj\OneDrive\Skrivebord\SampleVideo_720x480_1mb.mp4"
original_frame_folder = r"C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Frames"
video_to_frames(input_video, original_frame_folder)

#processing frames
processed_frame_folder = r"C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames"
process_frames(original_frame_folder, processed_frame_folder)

#rebuilding mp4
output_video_path = r"C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Output_vid\output.mp4"
frame_rate = 2
frames_to_video(processed_frame_folder, output_video_path, frame_rate)


Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Iteration 1: loss=174611.02
Saving C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames\ProcessedFrame-0001.png to C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames
Iteration 2: loss=174362.59
Saving C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames\ProcessedFrame-0001.png to C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames
Iteration 3: loss=174105.25
Saving C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames\ProcessedFrame-0001.png to C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames
frame0001.jpg
Iteration 1: loss=174559.89
Saving C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames\ProcessedFrame-0002.png to C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames
Iteration 2: loss=174311.86
Saving C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames\ProcessedFrame-0002.png to C:\Users\madsj\OneDrive\Uni\Deep_Learning\exam\Processed_Frames
Iteration 3: loss=174055.08
Saving C

### Wrapping up